In [ ]:
!pip install overpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for overpy: filename=overpy-0.6-py3-none-any.whl size=14091 sha256=266fbb7c4a7f4ca21755ae99c457e48308be8ec09559afd55150de7ee0368e44
  Stored in directory: /root/.cache/pip/wheels/6f/05/ef/529f51b5983091d5b53077e4f2f81d052a939573c5dd07acfc
Successfully built overpy


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import requests
import time
import pandas as pd
import overpy

In [ ]:
# Define the zip codes
zip_codes = ["3070", "3071", "3072", "3073", "3074", "3075", "3076", "3078", "3079", "3081",
             "3082", "3083", "3084", "3085", "3087", "3088", "3089", "3090", "3091", "3093",
             "3094", "3095", "3096", "3097", "3099", "3101", "3102", "3103", "3104", "3105",
             "3106", "3107", "3108", "3109", "3111", "3113", "3114", "3115", "3116", "3121",
             "3122", "3123", "3124", "3125", "3126", "3127", "3128", "3129", "3130", "3131",
             "3132", "3133", "3134", "3135", "3136", "3137", "3138", "3139", "3140", "3141"]

In [ ]:
# Define the amenities you are interested in
amenities = ["restaurant", "cafe", "bar", "library", "gym", "bookstore", "place_of_worship", "park", "churches", "public library", "club", "bookstore"]

In [ ]:
# Store the bounding box results
bounding_boxes = {}

In [ ]:
# Split the ZIP codes into chunks of 10
chunks = [zip_codes[i:i+10] for i in range(0, len(zip_codes), 10)]

In [ ]:
# Iterate over each chunk of ZIP codes
for chunk in chunks:
    for zip_code in chunk:
        url = f"https://nominatim.openstreetmap.org/search?q={zip_code}, Australia&format=json&polygon_geojson=1"
        response = requests.get(url)
        if response.status_code == 200 and len(response.json()) > 0:
            data = response.json()[0]
            bbox = data.get('boundingbox')
            if bbox is not None and len(bbox) == 4:
                lat1, lat2, lon1, lon2 = [float(coord) for coord in bbox]
                bounding_boxes[zip_code] = [lat1, lon1, lat2, lon2]
        time.sleep(1)  # Delay of 1 second between each request

In [ ]:
# Print the bounding boxes
for zip_code, bbox in bounding_boxes.items():
    lat1, lon1, lat2, lon2 = bbox
    print(f"ZIP code {zip_code} - Bounding box: {lat1},{lon1},{lat2},{lon2}")


ZIP code 3070 - Bounding box: -37.938449705882,144.84226109963,-37.618449705882,145.16226109963
ZIP code 3071 - Bounding box: -37.915860412439,144.84188572122,-37.595860412439,145.16188572122
ZIP code 3072 - Bounding box: -37.901411833197,144.84557284775,-37.581411833197,145.16557284775
ZIP code 3073 - Bounding box: -37.87539902406,144.85245971203,-37.55539902406,145.17245971203
ZIP code 3074 - Bounding box: -37.847970718519,144.85974617037,-37.527970718519,145.17974617037
ZIP code 3075 - Bounding box: -37.822103255,144.860236135,-37.502103255,145.180236135
ZIP code 3076 - Bounding box: -37.80026761,144.85938976,-37.48026761,145.17938976
ZIP code 3078 - Bounding box: -37.937291164433,144.85822534948,-37.617291164433,145.17822534948
ZIP code 3079 - Bounding box: -37.928388598138,144.89141222563,-37.608388598138,145.21141222563
ZIP code 3081 - Bounding box: -37.895697186555,144.88969788739,-37.575697186555,145.20969788739
ZIP code 3082 - Bounding box: -37.829144785789,144.90514026842,-37

In [ ]:
# Prepare the DataFrame to store the results
df = pd.DataFrame(columns=["id", "lat", "lon", "amenity", "name", "zip_code"])

In [ ]:
# Instantiate the Overpass API object
api = overpy.Overpass()

In [ ]:
# Iterate over each ZIP code
for zip_code, bbox in bounding_boxes.items():
    # Iterate over each amenity
    for amenity in amenities:
        # Fetch data about the amenity in the bounding box area
        result = api.query(f"""
        [out:json];
        (
          node["amenity"="{amenity}"]({','.join(map(str, bbox))});
          way["amenity"="{amenity}"]({','.join(map(str, bbox))});
          rel["amenity"="{amenity}"]({','.join(map(str, bbox))});
        );
        out center;
        """)

        # Process the result and add it to the DataFrame
        for node in result.nodes:
            df = df.append({
                'id': node.id,
                'lat': node.lat,
                'lon': node.lon,
                'amenity': amenity,
                'name': node.tags.get('name'),
                'zip_code': zip_code
            }, ignore_index=True)

        # Add a delay of 1 second between each request
        time.sleep(1)

Streaming output truncated to the last 5000 lines.
<ipython-input-12-c69aec7b7c3e>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-12-c69aec7b7c3e>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-12-c69aec7b7c3e>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-12-c69aec7b7c3e>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-12-c69aec7b7c3e>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-

In [ ]:
# Show the DataFrame
print(df)

                 id          lat          lon           amenity  \
0         261346373  -37.8534257  145.1605621        restaurant   
1         299026704  -37.8045430  145.0107596        restaurant   
2         308435480  -37.8335060  144.9669130        restaurant   
3         312967833  -37.8165887  144.8894117        restaurant   
4         323914661  -37.8417824  144.9672735        restaurant   
...             ...          ...          ...               ...   
167121   9903079303  -37.8232265  145.0790708  place_of_worship   
167122  10172773690  -37.9288687  145.1026316  place_of_worship   
167123  10695858489  -37.8040905  145.0833601  place_of_worship   
167124  10796831305  -37.7669890  145.1062068  place_of_worship   
167125  10809272188  -37.7900556  145.0899935  place_of_worship   

                                                     name zip_code  
0                                                   Sofia     3070  
1                                    Boulevard Restaurant

In [ ]:
# Save the DataFrame as a CSV file
df.to_csv('data.csv', index=False)

# Save the DataFrame as an Excel file
df.to_excel('data.xlsx', index=False)

In [ ]:
# Define the directory path where you want to save the files
output_directory = '/content/drive/MyDrive/Colab Notebooks/design/Untitled Folder/'

In [ ]:
import os

In [ ]:
# Save the DataFrame as a CSV file in the specified directory
csv_path = os.path.join(output_directory, 'data.csv')
df.to_csv(csv_path, index=False)
print(f"CSV file saved at: {csv_path}")

# Save the DataFrame as an Excel file in the specified directory
excel_path = os.path.join(output_directory, 'data.xlsx')
df.to_excel(excel_path, index=False)
print(f"Excel file saved at: {excel_path}")

CSV file saved at: /content/drive/MyDrive/Colab Notebooks/design/Untitled Folder/data.csv
Excel file saved at: /content/drive/MyDrive/Colab Notebooks/design/Untitled Folder/data.xlsx
